## UDTConverter

This converts files from the SSA-Format (PERIN-Format) to the format for annotation in the [Universal Data Tool](https://udt.dev/).

Should also be able to convert the UDT format back to the SSA-Format for evaluation.

In [59]:
# imports
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GroupShuffleSplit
import numpy as np
import os
import pandas as pd
import json
import uuid

In [70]:
# makedirs if not exist
os.makedirs("../../etl/data/processed/UDTConverter", exist_ok=True)

In [71]:
os.makedirs("../../etl/data/raw/UDTConverter", exist_ok=True)

In [61]:
# parameters
# Input data in SSA format (in JSON)
INPUT_DATA_SSA="../../etl/data/processed/Perin_Preprocessing/01_test.json"
UDT_OUTPUT_PATH="../../etl/data/processed/UDTConverter/01_test_udt.json"
UDT_INPUT_PATH="../../etl/data/raw/UDTConverter/01_test_udt_annotated.json"
OUTPUT_DATA_SSA="../../etl/data/processed/UDTConverter/01_test_ssa_annotated.json"
RANDOM_STATE=42

In [62]:
def read_json(INP_FILE):
    with open(INP_FILE, encoding="utf-8") as f:
        data = json.load(f)
        return data
    
ssa_data = read_json(INPUT_DATA_SSA)

In [63]:
def save_dict_as_json(dict, filename):
    with open(filename, 'w') as f:
        json.dump(dict, f)

In [64]:
# validate the correctness of the parsing
ssa_data[0]['opinions'][0]['Source'][0][0]

'Mann'

In [68]:
# check input format
ssa_data

[{'sent_id': '0',
  'text': 'Ein Mann schiesst am Weihnachtstag in einer Wohnung auf seine Freundin .',
  'opinions': [{'Source': [['Mann'], ['4:8']],
    'Target': [['Freundin'], ['62:70']],
    'Polar_expression': [['schiesst'], ['9:17']],
    'Polarity': 'Negative',
    'Intensity': 'Average'}]},
 {'sent_id': '1',
  'text': 'Zudem will sich die FPÖ stärker der Familienpolitik und hier insbesondere den Bedürfnissen von Alleinerziehenden widmen .',
  'opinions': [{'Source': [['FPÖ'], ['20:23']],
    'Target': [['Familienpolitik'], ['36:51']],
    'Polar_expression': [['widmen'], ['113:119']],
    'Polarity': 'Positive',
    'Intensity': 'Average'}]},
 {'sent_id': '2',
  'text': 'Die frühere Fraktionschefin und heutige Nationalrätin wird für die Stadionvorlage des Stadtrats eintreten .',
  'opinions': [{'Source': [['Fraktionschefin'], ['12:27']],
    'Target': [['Stadionvorlage'], ['67:81']],
    'Polar_expression': [['eintreten'], ['96:105']],
    'Polarity': 'Positive',
    'Intensit

In [50]:
def convert_ssa_to_udt(data: list, interface):
    for sent in data:
        # validate that...
        # has 1 opinion
        sample = {
            "_id": sent["sent_id"],
            "document": sent["text"],
        }
        try:
            len(sent['opinions']) == 1, "Sentences for annotation can currently only contain a single opinion."
            for opinion in sent['opinions']:
                for k, v in opinion.items():
                    print(k, v)
                # from IPython.core.debugger import Pdb; Pdb().set_trace()
                sample["annotation"] = {
                    "entities": [
                    {
                        "text": opinion["Polar_expression"][0][0],
                        "textId": str(uuid.uuid4().hex),
                        "label": "PolEx",
                        "start": int(opinion["Polar_expression"][1][0].split(":")[0]),
                        "end": int(opinion["Polar_expression"][1][0].split(":")[1])
                      },
                    ]
                }
        except Exception as e:
            print(f"Skipped converting to UDT, {e}")
            continue
        interface["samples"].append(sample)
    return interface

In [51]:
udt_dict = {
    "name": "Sentiment Dataset",
    "interface": {
    "type": "text_entity_relations",
    "entityLabels": [
      {
        "id": "Source",
        "displayName": "Source",
        "description": "The source of a sentiment or action."
      },
      {
        "id": "Target",
        "displayName": "Target",
        "description": "The target of a sentiment or action."
      },
      {
        "id": "Polar_expression",
        "displayName": "PolEx",
        "description": "A verb indicating a polar expression between a holder and a target?"
      }
    ],
    "relationLabels": [
      {
        "id": "Positive",
        "displayName": "Pro",
        "description": "A positive relation between a holder and a target."
      },
      {
        "id": "Neutral",
        "displayName": "Neutral",
        "description": "A neutral relation between a holder and a target."
      },
      {
        "id": "Negative",
        "displayName": "Con",
        "description": "A contra relation between a holder and a target."
      },
    ]
  },
  "samples": [],
}

In [52]:
udt_data = convert_ssa_to_udt(ssa_data, udt_dict)

Source [['Mann'], ['4:8']]
Target [['Freundin'], ['62:70']]
Polar_expression [['schiesst'], ['9:17']]
Polarity Negative
Intensity Average
Source [['FPÖ'], ['20:23']]
Target [['Familienpolitik'], ['36:51']]
Polar_expression [['widmen'], ['113:119']]
Polarity Positive
Intensity Average
Source [['Fraktionschefin'], ['12:27']]
Target [['Stadionvorlage'], ['67:81']]
Polar_expression [['eintreten'], ['96:105']]
Polarity Positive
Intensity Average
Source [['Nachbarn'], ['59:67']]
Target [['Lärmbelästigung'], ['105:120']]
Polar_expression [['beschwerten'], ['121:132']]
Polarity Negative
Intensity Average
Source [['Ungarn'], ['0:6']]
Target [['Zugang'], ['15:21']]
Polar_expression [['abgeschafft'], ['48:59']]
Polarity Negative
Intensity Average
Source [['Präsident'], ['42:51']]
Target [['Vergeltungsschlag'], ['105:122']]
Polar_expression [['brach'], ['23:28']]
Polarity Negative
Intensity Average
Source [['Christchindli-Märt'], ['4:22']]
Target [['6.'], ['51:53']]
Polar_expression [['findet'], [

In [53]:
# save as json file
save_dict_as_json(udt_data, UDT_OUTPUT_PATH)
udt_data

{'name': 'Sentiment Dataset',
 'interface': {'type': 'text_entity_relations',
  'entityLabels': [{'id': 'Source',
    'displayName': 'Source',
    'description': 'The source of a sentiment or action.'},
   {'id': 'Target',
    'displayName': 'Target',
    'description': 'The target of a sentiment or action.'},
   {'id': 'Polar_expression',
    'displayName': 'PolEx',
    'description': 'A verb indicating a polar expression between a holder and a target?'}],
  'relationLabels': [{'id': 'Positive',
    'displayName': 'Pro',
    'description': 'A positive relation between a holder and a target.'},
   {'id': 'Neutral',
    'displayName': 'Neutral',
    'description': 'A neutral relation between a holder and a target.'},
   {'id': 'Negative',
    'displayName': 'Con',
    'description': 'A contra relation between a holder and a target.'}]},
 'samples': [{'_id': '0',
   'document': 'Ein Mann schiesst am Weihnachtstag in einer Wohnung auf seine Freundin .',
   'annotation': {'entities': [{'te

### Output-Format / Input-Format

```json
{
  "name": "New undefined Dataset",
  "interface": {
    "type": "text_entity_relations",
    "entityLabels": [
      {
        "id": "food",
        "displayName": "Food",
        "description": "Edible item."
      },
      {
        "id": "hat",
        "displayName": "Hat",
        "description": "Something worn on the head."
      }
    ],
    "relationLabels": [
      {
        "id": "subject",
        "displayName": "Subject"
      }
    ]
  },
  "samples": [
    {
      "_id": "seqapblyl",
      "document": "I love you.",
      "annotation": {
        "entities": [
          {
            "text": "love",
            "textId": "yruukn",
            "start": 2,
            "end": 5
          }
        ],
        "relations": [
          {
            "from": "yruukn",
            "to": "yruukn",
            "label": "subject",
            "color": "#d32f2f"
          }
        ]
      },
      "brush": "complete"
    },
    {
      "_id": "scec120d4",
      "document": "I don't love you.",
      "annotation": {
        "entities": [
          {
            "text": "I don",
            "textId": "d3xn6k",
            "label": "food",
            "start": 0,
            "end": 4
          },
          {
            "text": "love you",
            "textId": "xkw8je",
            "label": "hat",
            "start": 8,
            "end": 15
          }
        ],
        "relations": [
          {
            "from": "d3xn6k",
            "to": "xkw8je",
            "label": "subject",
            "color": "#d32f2f"
          }
        ]
      },
      "brush": "complete"
    },
```


In [54]:
def convert_udt_to_ssa(input_list):
    output_list = []
    for dic in input_list:
        sent_id = dic['_id']
        text = dic['document']
        annotations = dic['annotation']
        entities = annotations['entities']
        relations = annotations['relations']
        opinions = []
        for rel in relations:
            opinion = {}
            for ent in entities:
                if ent['label'] == "PolEx":
                    opinion['Polar_expression'] = [[ent['text'], f"{ent['start']}:{ent['end']}"]]
                if ent['textId'] == rel['from']:
                    opinion['Source'] = [[ent['text'], f"{ent['start']}:{ent['end']}"]]
                if ent['textId'] == rel['to']:
                    opinion['Target'] = [[ent['text'], f"{ent['start']}:{ent['end']}"]]
            opinion['Polarity'] = rel['label']
            opinion['Intensity'] = "Average"
            opinions.append(opinion)
        output_dic = {'sent_id': sent_id, 'text': text, 'opinions': opinions}
        output_list.append(output_dic)
    print(output_list)
    return output_list

In [56]:
# only the samples dict!
udt_data = {
  "name": "Sentiment Dataset",
  "interface": {
    "type": "text_entity_relations",
    "entityLabels": [
      {
        "id": "Source",
        "displayName": "Source",
        "description": "The source of a sentiment or action."
      },
      {
        "id": "Target",
        "displayName": "Target",
        "description": "The target of a sentiment or action."
      },
      {
        "id": "Polar_expression",
        "displayName": "PolEx",
        "description": "A verb indicating a polar expression between a holder and a target?"
      }
    ],
    "relationLabels": [
      {
        "id": "Positive",
        "displayName": "Pro",
        "description": "A positive relation between a holder and a target."
      },
      {
        "id": "Neutral",
        "displayName": "Neutral",
        "description": "A neutral relation between a holder and a target."
      },
      {
        "id": "Negative",
        "displayName": "Con",
        "description": "A contra relation between a holder and a target."
      }
    ]
  },
  "samples": [
    {
      "_id": "0",
      "document": "Ein Mann schiesst am Weihnachtstag in einer Wohnung auf seine Freundin .",
      "annotation": {
        "entities": [
          {
            "text": "Mann",
            "textId": "h2d5vl",
            "label": "Source",
            "start": 4,
            "end": 7
          },
          {
            "text": "schiesst",
            "textId": "7a443915c23c414bb1c4e7bd5b2c02cb",
            "label": "PolEx",
            "start": 9,
            "end": 16
          },
          {
            "text": "Freundin",
            "textId": "9m8apf",
            "label": "Target",
            "start": 62,
            "end": 69
          }
        ],
        "relations": [
          {
            "from": "h2d5vl",
            "to": "9m8apf",
            "label": "Negative",
            "color": "#7b1fa2"
          }
        ]
      },
      "brush": "complete"
    },
    {
      "_id": "1",
      "document": "Zudem will sich die FPÖ stärker der Familienpolitik und hier insbesondere den Bedürfnissen von Alleinerziehenden widmen .",
      "annotation": {
        "entities": [
          {
            "text": "FPÖ",
            "textId": "mqc6yl",
            "label": "Source",
            "start": 20,
            "end": 22
          },
          {
            "text": "Bedürfnissen",
            "textId": "w87h93",
            "label": "Target",
            "start": 78,
            "end": 89
          },
          {
            "text": "widmen",
            "textId": "be3067d496e943408d5dda49e7898081",
            "label": "PolEx",
            "start": 113,
            "end": 118
          }
        ],
        "relations": [
          {
            "from": "mqc6yl",
            "to": "w87h93",
            "label": "Positive",
            "color": "#d32f2f"
          }
        ]
      },
      "brush": "complete"
    },
    {
      "_id": "2",
      "document": "Die frühere Fraktionschefin und heutige Nationalrätin wird für die Stadionvorlage des Stadtrats eintreten .",
      "annotation": {
        "entities": [
          {
            "text": "Fraktionschefin",
            "textId": "ryz5wk",
            "label": "Source",
            "start": 12,
            "end": 26
          },
          {
            "text": "Stadionvorlage",
            "textId": "w3we1e",
            "label": "Target",
            "start": 67,
            "end": 80
          },
          {
            "text": "eintreten",
            "textId": "f0bf40c9a5914de093461890807e756c",
            "label": "PolEx",
            "start": 96,
            "end": 104
          }
        ],
        "relations": [
          {
            "from": "ryz5wk",
            "to": "w3we1e",
            "label": "Positive",
            "color": "#d32f2f"
          }
        ]
      },
      "brush": "complete"
    },
  ]
}

# udt_data = read_json(UDT_INPUT_PATH)

In [57]:
udt_data = udt_data["samples"]
ssa_data = convert_udt_to_ssa(udt_data)

[{'sent_id': '0', 'text': 'Ein Mann schiesst am Weihnachtstag in einer Wohnung auf seine Freundin .', 'opinions': [{'Source': [['Mann', '4:7']], 'Polar_expression': [['schiesst', '9:16']], 'Target': [['Freundin', '62:69']], 'Polarity': 'Negative', 'Intensity': 'Average'}]}, {'sent_id': '1', 'text': 'Zudem will sich die FPÖ stärker der Familienpolitik und hier insbesondere den Bedürfnissen von Alleinerziehenden widmen .', 'opinions': [{'Source': [['FPÖ', '20:22']], 'Target': [['Bedürfnissen', '78:89']], 'Polar_expression': [['widmen', '113:118']], 'Polarity': 'Positive', 'Intensity': 'Average'}]}, {'sent_id': '2', 'text': 'Die frühere Fraktionschefin und heutige Nationalrätin wird für die Stadionvorlage des Stadtrats eintreten .', 'opinions': [{'Source': [['Fraktionschefin', '12:26']], 'Target': [['Stadionvorlage', '67:80']], 'Polar_expression': [['eintreten', '96:104']], 'Polarity': 'Positive', 'Intensity': 'Average'}]}]


In [69]:
ssa_data

[{'sent_id': '0',
  'text': 'Ein Mann schiesst am Weihnachtstag in einer Wohnung auf seine Freundin .',
  'opinions': [{'Source': [['Mann'], ['4:8']],
    'Target': [['Freundin'], ['62:70']],
    'Polar_expression': [['schiesst'], ['9:17']],
    'Polarity': 'Negative',
    'Intensity': 'Average'}]},
 {'sent_id': '1',
  'text': 'Zudem will sich die FPÖ stärker der Familienpolitik und hier insbesondere den Bedürfnissen von Alleinerziehenden widmen .',
  'opinions': [{'Source': [['FPÖ'], ['20:23']],
    'Target': [['Familienpolitik'], ['36:51']],
    'Polar_expression': [['widmen'], ['113:119']],
    'Polarity': 'Positive',
    'Intensity': 'Average'}]},
 {'sent_id': '2',
  'text': 'Die frühere Fraktionschefin und heutige Nationalrätin wird für die Stadionvorlage des Stadtrats eintreten .',
  'opinions': [{'Source': [['Fraktionschefin'], ['12:27']],
    'Target': [['Stadionvorlage'], ['67:81']],
    'Polar_expression': [['eintreten'], ['96:105']],
    'Polarity': 'Positive',
    'Intensit